# Little, Big Data using Colaboratory
## For use with "LittleBigData.py"
### Emily Stark
### April, 30th, 2019

This script is intended for use to analyze datasets with restricted sample size and high dimensional data. In order to complete this analysis, you must have LittleBigData.py in the same folder as this script. Your data do not need to be in the same directory, but the directory must be known.

In [ ]:
from google.colab import drive 
drive.mount('/content/drive') # mount drive

In [ ]:
import os
import numpy as np

In [ ]:
os.chdir('/content/drive/...')
os.listdir()

In [ ]:
 from LittleBigData import *

## List of Functions:

*Preproc(x, w, Xstart, Xend, Wstart, Wend, trainfiledir, valfiledir, classtargets, header, testfiledir = None)*
 - **x** (integer), how many rows each input contains
 
 - **w** (integer), how many columns each input contains
 
 - **Xstart**, Xend, Wstart, Wend (integer), which row/column to start/end analysis, used for cropping
 
 - **trainfiledir**, **valfiledir**, **testfiledir** (string), file path to location of .csv files of data for training, validation, and testing (if provided)
 
 - **classtargets** (list), the marker in the file name that denotes ground truth label
 
 - **header** (interger), number of rows to skip in the .csv file of data
 
***

*Modeltrain(Xtrain, Ytrain, Xval, Yval, mname, net = '5FCN', ep = 80, bsize = 50)*
 - **Xtrain**, **Xval** (array), 4D tensor containing data with the first dimension representing inputs, second representing rows for each input, third represnting columns for each input, and fourth representing channels for each input (usually only 1).
 
 - **Ytrain**, **Yval** (array), one-hot vector containing ground truth labels for each input.
 
 - **mname** (string), name of the model for tensorboard and to save progress.
 
 - **net** (string or specified architecture), can choose '3FCN', '5FCN', or 'AlexNet' to select prewritten architectures or write your own and specify the variable name here. Default is 5FCN.
 
 - **ep** (integer), number of epochs. Defaul 80.
 
 - **bsize** (integer), batch size. Default 50.
 
 - - **OS** (string), either 'Colab' or 'Linux' used to show Tensorboard. If running on Windows, do not specify anything, defaults to None
 
***

*DeepDiscovery(Xval, Yval, classnames, n, modeldir = None, mname = None, model = None, net = '5FCN')*
 - **classnames** (list), character strings representing class names in the order of representation in the one-hot vector.
 
 - **n** (integer), number of inputs per sample.
 
 - **modeldir** (string) and **mname** (string), file path to the saved model and the model name to load in weights. Must either include this or model.
 
 - **model** (trained model), variable for the model already loaded into the jupyter notebook.
 
***

*GestaltDL(Xval, Yval, valnames, classnames, n, perc,  Xtest = None, Ytest = None, testnames = None, modeldir = None, mname = None, model = None, net = '5FCN')*
 - **valnames**/**testnames** (list), names of the samples in the validation/testing set (generally use file names).
 
 - **perc** (integer), number between 0 and 100 of the percentile of interest for identifying strong signals (i.e. if looking for the top 10% of signals, choose 90).
 


## Completing Little, Big Data Analysis
Use the following blocks of code to complete the preprocessing, model training, and postprocessing. Once the model is trained, DeepDiscovery will show where in the sample the model is confident in the correct classification across all classes. You may decide to crop to that region or continue on to GestaltDL.

In [ ]:
## Code to save my valnames and testnames
import os
import glob

os.chdir('...')
valnames = glob.glob('*csv')
os.chdir('...')
testnames = glob.glob('*csv')

## Preprocessing

In [ ]:
Xtrain, Ytrain, Xval, Yval, Xtest, Ytest = Preproc(x = 1000, w = 100, Xstart = 0, Xend = 1000, 
                                                   Wstart = 0, Wend = 5000, 
        trainfiledir = '.../Train', 
        valfiledir = '.../Val', 
        testfiledir = '.../Test', 
        classtargets = ['A','B'], header = 0)

It is recommended that you save your tensors as .npy files so you can load them in without having to preprocess your data again. To do so, change your directory and use the np.save commands below.

In [ ]:
os.chdir('.../NumpyData')
np.save('Xtrain.npy', Xtrain)
np.save('Ytrain.npy', Ytrain)
np.save('Xval.npy', Xval)
np.save('Yval.npy', Yval)
np.save('Xtest.npy', Xtest)
np.save('Ytest.npy', Ytest)

If you want to train a different network, you can copy the code below:

os.chdir('/content/drive/My Drive/Olfaction/Dissemination/Presentations/2019.05.26 ISOEN Tutorial/FvsChi_Data/NumpyData')

Xtrain = np.load('Xtrain.npy') ...

## Specifying New Achitecture
If you do not want to use one of the preloaded architectures (3-hidden layer fully connected, 5-hidden layer fully connected, or AlexNet) you are welcome to specify your own using TFLearn/Tensorflow. An example of a 1-hidden layer fully connected network is shown below. It is recommendended not to alter the input or output layer, as these will ensure your network is properly set up to take in the appropriately sized data and output the correct number of probabilistic values. When you have written your architecture, instead of including one of the three specified strings for "net" in *Modeltrain*, just include the variable name for your network (in the example below, "mynet").

If you want to write your own network, you need to import necessary functions/libraries explicitly.

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical

import tensorflow as tf

In [ ]:
mynet = input_data(shape = [None, Xtrain.shape[1], Xtrain.shape[2], Xtrain.shape[3]])
mynet = fully_connected(mynet, 2000, activation='tanh')
mynet = fully_connected(mynet, Ytrain.shape[1], activation='softmax')
mynet = regression(mynet, optimizer='momentum',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

## Model Training
To save your model in a specific location, before training your model you will need to change directory.

In [ ]:
os.chdir('.../Model')

In [ ]:
mymodel = Modeltrain(Xtrain = Xtrain, Ytrain = Ytrain, Xval = Xval, Yval = Yval, 
           mname = 'testestest', arch = mynet, ep = 2, bsize = 100, OS = 'Colab')

To save your trained model, it is simple.

In [ ]:
mymodel.save('mymodel_testestest123')

The functions below will work with either saved models or with models that you have trained during the runtime of this notebook. If you load in the saved model, you will use the "modeldir" and "mname" arguments. If you use a model that is already loaded into the notebook (for example, if you just trained it) you will use the "model" argument.

## Deep Discovery

In [ ]:
DeepDiscovery(Xval = Xval, Yval = Yval, classnames = ['A', 'B'], 
              n = 50, model = mymodel, net = mynet)

## Gestalt Deep Learning

In [ ]:
GestaltDL(Xval = Xval, Yval = Yval, valnames = valnames, classnames = ['A', 'B'],
          n = 50, perc = 90,  Xtest = Xtest, Ytest = Ytest, testnames = testnames, 
          model = mymodel, net = mynet)